In [14]:
import pandas as pd
import numpy as np
import os

# --- 1. GOOGLE DRIVE MOUNT ---
try:
    from google.colab import drive
    drive.mount('/content/drive')
    log_prefix = "/content/drive/MyDrive/Capstone/data/"
except:
    log_prefix = "" # Fallback for local execution


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# --- 2. CONFIGURATION & AUDIT LOGGING ---
def log_audit(message):
    print(f"[AUDIT LOG] {message}")

INTERNAL_DATA_PATH = os.path.join(log_prefix, 'Internal_Bank_Dataset.xlsx')
EXTERNAL_DATA_PATH = os.path.join(log_prefix, 'External_Cibil_Dataset.xlsx')
OUTPUT_PATH = 'integrated_data.csv'


In [16]:
# --- 3. DATA LOADING ---
log_audit("Commencing data load...")

try:
    # Reading Excel files
    df_internal = pd.read_excel(INTERNAL_DATA_PATH)
    df_external = pd.read_excel(EXTERNAL_DATA_PATH)

    log_audit(f"SUCCESS: Internal Data Shape: {df_internal.shape}")
    log_audit(f"SUCCESS: External Data Shape: {df_external.shape}")
except FileNotFoundError:
    log_audit(f"ERROR: File not found at {INTERNAL_DATA_PATH}. Please check the folder path.")
    raise

[AUDIT LOG] Commencing data load...
[AUDIT LOG] SUCCESS: Internal Data Shape: (51336, 26)
[AUDIT LOG] SUCCESS: External Data Shape: (51336, 62)


In [17]:
# --- 4. DATA INTEGRITY CHECK: DUPLICATES ---
def check_duplicates(df, name):
    # Identifying duplicates based on Prospect ID
    duplicates = df.duplicated(subset=['PROSPECTID']).sum()
    log_audit(f"Duplicates found in {name}: {duplicates}")
    if duplicates > 0:
        df.drop_duplicates(subset=['PROSPECTID'], keep='first', inplace=True)
        log_audit(f"Duplicates removed from {name} to ensure unique primary keys.")
    return df

df_internal = check_duplicates(df_internal, "Internal Data")
df_external = check_duplicates(df_external, "External Data")


[AUDIT LOG] Duplicates found in Internal Data: 0
[AUDIT LOG] Duplicates found in External Data: 0


In [18]:
# --- 5. DATA INTEGRATION (INNER JOIN) ---
log_audit("Performing Inner Join on 'PROSPECTID'...")

# Merging both datasets
df_final = pd.merge(
    df_internal,
    df_external,
    on='PROSPECTID',
    how='inner'
)

log_audit(f"Rows after merge: {len(df_final)}")

[AUDIT LOG] Performing Inner Join on 'PROSPECTID'...
[AUDIT LOG] Rows after merge: 51336


In [19]:
# --- 6. DATA INTEGRITY AUDIT ---
missing_in_join = max(len(df_internal), len(df_external)) - len(df_final)
log_audit(f"Records dropped due to non-matching IDs: {missing_in_join}")

critical_cols = ['Approved_Flag', 'NETMONTHLYINCOME', 'Credit_Score']
available_cols = [col for col in critical_cols if col in df_final.columns]
log_audit(f"Critical columns verified: {available_cols}")

[AUDIT LOG] Records dropped due to non-matching IDs: 0
[AUDIT LOG] Critical columns verified: ['Approved_Flag', 'NETMONTHLYINCOME', 'Credit_Score']


In [20]:
# --- 7. EXPORT ---
df_final.to_csv(OUTPUT_PATH, index=False)
log_audit(f"Process Complete. Integrated dataset saved as {OUTPUT_PATH}.")

[AUDIT LOG] Process Complete. Integrated dataset saved as integrated_data.csv.
